# Description

Create summary from NCBI RAPT pipeline results files.

**IN:** Result folders from NCBI RAPT pipline with:
- assembly_stat_report.tsv
- annot.gbk
- ani-tax-report.xml

**Out:**
    CSV summary

**Author:** Edgars Liepa

**Email:** edgars.liepa@biomed.lu.lv

Developed at Latvian Biomedical Research and Study center

03.11.2022

In [21]:
# Get list of foldes in local directory with sample results
import os
for root, dirs, files in os.walk(".", topdown=False):
   # for name in files:
   #    print(os.path.join(root, name))
   for name in dirs:
      print(os.path.join(root, name))

./.git/logs/refs/remotes/origin
./.git/logs/refs/heads
./.git/logs/refs/remotes
./.git/logs/refs
./.git/objects/pack
./.git/objects/20
./.git/objects/info
./.git/objects/22
./.git/objects/d3
./.git/refs/remotes/origin
./.git/refs/heads
./.git/refs/remotes
./.git/refs/tags
./.git/logs
./.git/hooks
./.git/branches
./.git/objects
./.git/info
./.git/refs
./557259f-w06-e272_output
./41c95d3-5sy-4c24_output
./9095857-87h-bf63_output
./1c93b92-daa-0119_output
./c6376e7-zs3-f15e_output
./9f88d34-den-f79d_output
./14030d9-c6s-2d8b_output
./0d0a7f5-ohp-5f3f_output
./299ef09-kfr-1d26_output
./20670e9-ygs-94d0_output
./65e1f2c-266-3d1c_output
./2bccc4b-h24-b74f_output
./9c12216-8k1-c82a_output
./ceeec5e-wku-15fe_output
./95ed6de-u4q-0073_output
./ebada13-1sc-998b_output
./1a7b916-oyi-74d4_output
./584b677-23z-4771_output
./4284c3c-28h-6025_output
./3dc10cd-rma-91cd_output
./bab69c5-y4f-1d90_output
./9bbd667-cdx-18d2_output
./4064a86-wfd-7fdf_output
./4a77690-ln9-205f_output
./06543a2-xjb-58b4_outp

## Get assembly stats
From assembly_stat_report.tsv get:
- Total sequence count

In [9]:
import pandas as pd

def assembly(root, name):

    assembly_stat = pd.read_csv(os.path.join(root, name)+'/assembly_stat_report.tsv', sep="	")

    return assembly_stat["Total_seqs"][0], assembly_stat["Total_len(bp)"][0]


## Get longest conting

In [10]:
def getContig(root, name, longest_contig):

    from Bio import SeqIO
    
    for seq_record in SeqIO.parse(os.path.join(root, name)+'/annot.gbk', "genbank"):
        if (longest_contig == len(seq_record)):
            return seq_record
    
    return 0

## Get predicted taxa

Parse ani-tax-report.xml from NCBI RAPT results and get predicted bacteria taxonomical name

In [11]:
from lxml import etree

def getTaxa(root, name):

    if ".git/" in os.path.join(root, name):
            return
    tree = etree.parse(os.path.join(root, name)+'/ani-tax-report.xml')

    # print(len(tree.error_log))
    tree.getroot()

    result = etree.tostring(tree.getroot(),pretty_print=True, method="html")
    result

    submitted_taxid = tree.getroot()[0][0]
    predicted_taxid = tree.getroot()[0][1]
    
    return submitted_taxid, predicted_taxid, tree.getroot()[1][0].get("ANI"), tree.getroot()[1][0].get("query_pct_coverage")

## Create summary files

In [29]:
# Print Results
import csv
import sys
import os

header = ['Sample Name','predicted_taxid (NCBI)', 'submitted_taxid (Kraken)', 'Predicted taxa confidance', 'Average Nucleotide Identity (ANI)' , "query_pct_coverage", 'Conting Count', 'Total Sequence Length',] # 'CDSs', 'rRNAs']

# create DataFrame with samples names and result ID_s
file_names = pd.read_csv('NCBI_rez_names.csv', sep=",")

with open('anotationStat.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

    for root, dirs, files in os.walk(".", topdown=False):
        for name in dirs:

            # os.walk includes .git subdirectories. Only RAPT result folders needed, so rest should be ignored
            if ".git" in os.path.join(root, name):
                continue

            print('parse:',os.path.join(root, name))
            
            submitted_taxid, predicted_taxid, ani, query_pct_coverage = getTaxa(root, name)
            print(submitted_taxid.tag, ":", submitted_taxid.get("org-name"), "({})".format(submitted_taxid.get("rank")))
            print(predicted_taxid.tag, ":", predicted_taxid.get("org-name"), "({})".format(predicted_taxid.get("rank")))
            
            totelSeq, total_seq_len = assembly(root, name)

            print(totelSeq, total_seq_len)


            # seq_record = getContig(root, name, longest_contig)
            
            writer.writerow([file_names.loc[file_names['NCBI RAPT NAME'] == name]['SAMPLE NAME'].item(),predicted_taxid.get("org-name"), submitted_taxid.get("org-name"), predicted_taxid.get("confidence"), ani, query_pct_coverage, totelSeq, total_seq_len ])

            

parse: ./557259f-w06-e272_output
submitted-taxid : Bacillus pumilus (species)
predicted-taxid : Bacillus pumilus (species)
734 3781760
parse: ./41c95d3-5sy-4c24_output
submitted-taxid : Pseudomonas fluorescens (species)
predicted-taxid : Pseudomonas kilonensis (species)
581 6447964
parse: ./9095857-87h-bf63_output
submitted-taxid : Streptomyces (genus)
predicted-taxid : [Kitasatospora] papulosa (species)
2433 7258651
parse: ./1c93b92-daa-0119_output
submitted-taxid : Streptomyces (genus)
predicted-taxid : Streptomyces albidoflavus (species)
2769 6595709
parse: ./c6376e7-zs3-f15e_output
submitted-taxid : Streptomyces (genus)
predicted-taxid : Streptomyces griseus (species)
1658 8249253
parse: ./9f88d34-den-f79d_output
submitted-taxid : Bacillus (genus)
predicted-taxid : Bacillus altitudinis (species)
151 3718059
parse: ./14030d9-c6s-2d8b_output
submitted-taxid : Bacillus (genus)
predicted-taxid : Bacillus subtilis (species)
214 4272100
parse: ./0d0a7f5-ohp-5f3f_output
submitted-taxid : 

## Get gene products

Parse .gbk file and extract Gene; CDS; rRNA; tRNA; ncRNA; products

In [2]:
import csv
import sys
import os
from Bio import SeqIO

# Set First flag to print organism name at the top of the file
first = True

for root, dirs, files in os.walk(".", topdown=False):    
    for name in dirs:
    
        # os.walk includes .git subdirectories. Only RAPT result folders needed, so rest should be ignored
        if ".git" in os.path.join(root, name):
            continue
    
        print('parse:',os.path.join(root, name))
        
        with open(os.path.join(root, name)+'/geneProducts.csv', 'w', encoding='UTF8', newline='') as f:
            
            writer = csv.writer(f)
            # writer.writerow(header)
            
            for record in SeqIO.parse(os.path.join(root, name)+'/annot.gbk', "genbank"):
                
                if (first == True):
                    writer.writerow([record.annotations['organism'], "GENES TOTAL: ", record.annotations['structured_comment']['Genome-Annotation-Data']['Genes (total)']])
                    first = False
                
                print(record.id)
                writer.writerow(["Contig ID",record.id ])
            
                for feature in record.features:
                    if ('product') in feature.qualifiers:
                        print(feature.type, feature.qualifiers['product'], feature.location)
                        writer.writerow([feature.type,feature.qualifiers['product'], feature.location])



NameError: name 'csv' is not defined

In [ ]:
record == start()

NameError: name 'start' is not defined

In [ ]:
for record in genome_record:
    # print(record.title)
    for feature in record.features:
        print(feature.id)

NameError: name 'genome_record' is not defined

In [ ]:
for record in genome_record:
    for feature in record.features:
        print(repr(feature))


In [ ]:
# feature location
# print(feature.location)

# ?feature.qualifiers

if 'product' in feature.type:
    print(feature.qualifiers['product'])

In [ ]:
print(feature.type)

source


In [ ]:
print(record.qualifiers)

AttributeError: 'SeqRecord' object has no attribute 'qualifiers'

In [ ]:
record.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(339), strand=1), type='source'),
 SeqFeature(FeatureLocation(BeforePosition(0), AfterPosition(339), strand=-1), type='gene'),
 SeqFeature(FeatureLocation(BeforePosition(0), AfterPosition(339), strand=-1), type='CDS')]

In [ ]:
record

SeqRecord(seq=Seq('TTTTATGCGTTTCATAATGCCTTGGATTGAAGATGAGCGGAAGTTATCAGAGCC...TAG'), id='Contig_172_5.47962', name='Contig_172_5.47962', description='Pseudomonas chromosome, whole genome shotgun sequence', dbxrefs=[])

In [ ]:
suk = SeqIO.parse(os.path.join(root, name)+'/annot.gbk', "genbank")

?suk

Type:           GenBankIterator
String form:    <Bio.SeqIO.InsdcIO.GenBankIterator object at 0x7f8efb2458a0>
File:           /usr/lib/python3/dist-packages/Bio/SeqIO/InsdcIO.py
Docstring:      Parser for GenBank files.
Init docstring:
Break up a Genbank file into SeqRecord objects.

Argument source is a file-like object opened in text mode or a path to a file.
Every section from the LOCUS line to the terminating // becomes
a single SeqRecord with associated annotation and features.

Note that for genomes or chromosomes, there is typically only
one record.

This gets called internally by Bio.SeqIO for the GenBank file format:

>>> from Bio import SeqIO
>>> for record in SeqIO.parse("GenBank/cor6_6.gb", "gb"):
...     print(record.id)
...
X55053.1
X62281.1
M81224.1
AJ237582.1
L31939.1
AF297471.1

Equivalently,

>>> with open("GenBank/cor6_6.gb") as handle:
...     for record in GenBankIterator(handle):
...         print(record.id)
...
X55053.1
X62281.1
M81224.1
AJ237582.1
L31939.1
AF2974

In [ ]:
from Bio import GenBank
with open(os.path.join(root, name)+'/annot.gbk') as handle:
    rec = GenBank.parse(handle)

    rec

TypeError: 'callable_iterator' object is not subscriptable

In [ ]:
record.annotations['structured_comment']['Genome-Annotation-Data']['Genes (total)']

'4,360'

In [2]:
from lxml import etree

tree = etree.parse("/home/edgars.liepa/Becteria result/0d0a7f5-ohp-5f3f_output/ani-tax-report.xml")

# print(len(tree.error_log))
tree.getroot()

result = etree.tostring(tree.getroot(),pretty_print=True, method="html")
result

submitted_taxid = tree.getroot()[0][0]
predicted_taxid = tree.getroot()[0][1]
    

In [10]:
print(predicted_taxid.get("confidence"))

HIGH


In [28]:
tree.getroot()[1][0].get("ANI")

'97.186785402870797'

In [28]:
import pandas as pd


file_names = pd.read_csv('NCBI_rez_names.csv', sep=",")

print(file_names.loc[file_names['NCBI RAPT NAME'] == '557259f-w06-e272_output']['SAMPLE NAME'].item())

VN10_Bacillus subtilis 73
